## ML Algorithms, Part 8, Naive Bayes


### Bayes Theorem

Naive Bayes is an algorithm that makes use of Bayes Theorem:

$ P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)} $

Without getting into the details of probabilities, all that Bayes Theorem tells us is that our ideas, beliefs, or perceptions should change based on new information or evidence and their strength or importance. Let's see an example:

> I am very positive (say 99%) that aliens don't exist. Let's say tomorrow, NASA, announces new discoveries that might mean there is extraterrestrial life. I'm still convinced that aliens don't exist (down to 98%). Over the next days RFSA (Russia), ESA (Europe), and CNSA (China), each announces similar, independent, discoveries. I'm still convinced of my initial belief, but following Bayes advice, now I'm willing to discuss the possibility (95%). 

> Fast forward a year, and we find structures on Mars! If I hadn't suspected a conspiracy (not being a fan of conspiracy theories, I update my belief that there is a conpiracy from 1% to 5%), or perhaps that our ancestors from 10k years ago visited Mars (again, not a fan, so from 0.01% to 0.1%), I'd be convinced that Martians exist (well, at least they are more likely than not, say 80% they do - or to compare with previous numbers, 20% convinced they don't exist)! However unlikely, all of these are definite possibilities since we lack complete information. 
    
> But how did I reach these numbers? A discovery that "might mean" existence of life, and a discovery of structures (concrete evidence) should not affect the change of my beliefs in the same way. Beside the absolute strength of evidence, we should also consider what other possible explanations could lead to the evidence, as well as how the evidence affects other hypotheses.
    
Calculating probabilities can be fun, and using distributions of probabilities even more so, but this article won't be enough to catch up on Bayesian Statistics. Instead, we can see how this simple theorem (paired with a 'naive' assumption) can be used.

### Gender Identification

The most widely known application of Naive Bayes is for filtering spam emails, but let's look at another example, and use NLTK (sklearn also has an implementation of Naive Bayes, you can try to use that as an exercise), a large library that specializes on Natural Language Processing. We will walk through the example from their documentation.

#### Imports

In [1]:
import nltk
from nltk.corpus import names # used as training data
from sklearn.model_selection import train_test_split

#### Feature extraction

There are many algorithms to extract features from text; most rely on vectorization. Some of these are the **Bag of Words** model, **Term Frequency - Inverse Document Frequency**, and **Embeddings**. Here we will use the first and last letter of a name as features.

In [2]:
def gender_features(word):
    return {'first_letter':word[0],'last_letter': word[-1]}

Let's see the features generated for Alexei ([by the way, if Computer Vision is your thing, take a look at Prof. Alexei Efros' work](http://people.eecs.berkeley.edu/~efros/)):

In [3]:
alexei = gender_features("Alexei")
alexei

{'first_letter': 'A', 'last_letter': 'i'}

In [4]:
# our dataset: a list of names and the respective gender labels
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
                 [(name, 'female') for name in names.words('female.txt')])

labeled_names[:3] + labeled_names[-3:]

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abbey', 'male'),
 ('Zsazsa', 'female'),
 ('Zulema', 'female'),
 ('Zuzana', 'female')]

In [5]:
# conversion of the list of data points to features
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]

In [6]:
# we only have 1 iterable
# the form differs from sklearn, but this is what NLTK expects
train_data, test_data = train_test_split(featuresets)

In [7]:
# Here's how we create a classifer, defined and fitted at the same time 
classifier = nltk.NaiveBayesClassifier.train(train_data)

#### Predict!

Same as with the training data, we need to convert the features, then call `classify` method of our classifier.

In [8]:
classifier.classify(alexei)

'female'

Our model could probably learn a thing or two from the professors work. Perhaps if we try another spelling?

In [9]:
classifier.classify(gender_features("Alexey"))

'female'

Our Slav/Greek friends will certainly be mad at our model for missclassifying them. Either our features are bad, or the algorithm is not the best. Although we won't answer this question in detail, both of these effects are to blame (ever wondered why so many non-spam emails end up in the junk folder?). In fact, Naive Bayes (NB) is usually the starting point and is usually combined with other models. 

Still, the model achieves better performance than random (see test set accuracy below). Considering it is a very simple model (did we say that it is pretty light on your memory as well?!), and it made a 'naive' assumption, we are pretty content. 

> But what is the naive assumption?

The "Naive" part of NB that we did not discuss so far has to do with the assumption that the existence of every word, or data point, is independent of the others. In the spam filter example, it would mean that the presence of the words "gold" and "sex" tells us nothing about the presence of the word "rolex" in the same email. Perhaps that is correct for this collection of words and for some emails, but is faulty for most cases. But it works. And quite well. That's why NB is an important algorithm to know.

In [10]:
# Often it is good to compare with a baseline, here
# since we have more women names, we are using a 
# classifier that assigns the label 'woman' to every name
baseline = sum(1 for x in test_data if x[1] == "female") / len(test_data)
print("Accuracy if we predicted only 'woman': {:.2f}%".format(baseline*100))

# Actual model accuracy
print("Test set accuracy: {:.2f}%".format(nltk.classify.accuracy(classifier, test_data)*100))

Accuracy if we predicted only 'woman': 63.24%
Test set accuracy: 77.09%


The [nltk article](https://www.nltk.org/book/ch06.html) continues into more depth from this point but that is outside our pursuits here. Before leaving you, we will demonstrate how to extract features with sklearn (so you can later model it, as suggested earlier).

Using sklearns `TfidfVectorizer` we provide a list of sentences, paragraphs, "documents", and we get a sparse matrix (a table with mostly zeros) where each row corresponds to each document. TF-IDF works by finding how often a term appears in a certain document in relation to how often it appears across documents ([view more here](https://en.wikipedia.org/wiki/Tf%E2%80%93idf))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
sentences = ["Sample sentence one",
            "Another sample sentence, two",
            "Sample sentence three is here"]

In [13]:
tfidf = TfidfVectorizer()

features = tfidf.fit_transform(sentences)
features # sparse results

<3x8 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [14]:
# but since we have a small matrix, we can view its dense version
features.todense()

matrix([[0.        , 0.        , 0.        , 0.76749457, 0.45329466,
         0.45329466, 0.        , 0.        ],
        [0.6088451 , 0.        , 0.        , 0.        , 0.35959372,
         0.35959372, 0.        , 0.6088451 ],
        [0.        , 0.52004008, 0.52004008, 0.        , 0.30714405,
         0.30714405, 0.52004008, 0.        ]])

And now with these feature vectors you can train your classifiers.